<a href="https://colab.research.google.com/github/AnjaDeric/MDA-TeamCroatia/blob/main/Data_Preparation_and_Pre_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
from datetime import date, timedelta, datetime
import requests
pd.options.mode.chained_assignment = None

# 1 Geographical County/State Data

In this section, we prepare a list of all states and counties we will be working with for our dashboard, including their longitude, latitude, and population. Since our dashboard will provide the user with the shortest and safest driving route, we focus on extracting only data for continental US (ecluding all territories, Hawaii, and Alaska).

Load the original info dataset from the Johns Hopkins GitHub page. This data includes all counties (including the US and all US territories). It will thus be filtered to extract only the necessary US state and county data.

In [ ]:
url="https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv"
geo_info = pd.read_csv(url)
geo_info.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
0,4,AF,AFG,4.0,NaN,NaN,NaN,Afghanistan,33.93911,67.709953,Afghanistan,38928341.0
1,8,AL,ALB,8.0,NaN,NaN,NaN,Albania,41.15330,20.168300,Albania,2877800.0
2,10,AQ,ATA,10.0,NaN,NaN,NaN,Antarctica,-71.94990,23.347000,Antarctica,NaN
3,12,DZ,DZA,12.0,NaN,NaN,NaN,Algeria,28.03390,1.659600,Algeria,43851043.0
4,20,AD,AND,20.0,NaN,NaN,NaN,Andorra,42.50630,1.521800,Andorra,77265.0


## 1.1 Prepare State-Level Data

From the full data set, we want to extract only US state data (excluding all US territories and similar). This can be done by filtering the original dataset and only keeping the rows where the FIPS value is between 01 (corresponding to Alabama) and 56 (corresponding to Wyoming).

In [ ]:
state_info = geo_info[geo_info['FIPS'].between(1, 56)]
state_info = state_info[['FIPS','Province_State','Lat','Long_','Population']]
state_info.rename(columns = {'FIPS':'fips', 'Province_State':'state',
                      'Lat':'lat', 'Long_':'long', 'Population':'population'}, inplace = True)
state_info.head()

,fips,state,lat,long,population
999,1.0,Alabama,32.3182,-86.9023,4903185.0
1000,2.0,Alaska,61.3707,-152.4044,731545.0
1001,4.0,Arizona,33.7298,-111.4312,7278717.0
1002,5.0,Arkansas,34.9697,-92.3731,3017804.0
1003,6.0,California,36.1162,-119.6816,39512223.0


Reformat FIPS to be a 2 character string.

In [ ]:
state_info.fips = (state_info.fips.astype(int)).astype(object)
state_info['fips'] = state_info['fips'].apply('{:0>2}'.format)
state_info.head()

,fips,state,lat,long,population
999,01,Alabama,32.3182,-86.9023,4903185.0
1000,02,Alaska,61.3707,-152.4044,731545.0
1001,04,Arizona,33.7298,-111.4312,7278717.0
1002,05,Arkansas,34.9697,-92.3731,3017804.0
1003,06,California,36.1162,-119.6816,39512223.0


Since we will be looking at safest routes to take during COVID, we only keep data for continental US -- that is, we remove Hawaii and Alaska from the data set (as one would need to fly in order to reach other states from Hawaii or Alaska, assuming they do not leave the US).

In [ ]:
state_info = state_info[(state_info.state != "Alaska") & (state_info.state != "Hawaii")]
state_info.reset_index(inplace=True, drop = True)

We complete one final check to ensure all rows correspond to states we want to keep in the dataset and all states are present.

In [ ]:
state_info

,fips,state,lat,long,population
0,01,Alabama,32.3182,-86.9023,4903185.0
1,04,Arizona,33.7298,-111.4312,7278717.0
2,05,Arkansas,34.9697,-92.3731,3017804.0
3,06,California,36.1162,-119.6816,39512223.0
4,08,Colorado,39.0598,-105.3111,5758736.0
5,09,Connecticut,41.5978,-72.7554,3565287.0
6,10,Delaware,39.3185,-75.5071,973764.0
7,11,District of Columbia,38.8974,-77.0268,705749.0
8,12,Florida,27.7663,-81.6868,21477737.0
9,13,Georgia,33.0406,-83.6431,10617423.0


## 1.2 Prepare County-Level Data

From the full data set, we want to extract only US county data (excluding all US territories and similar). This can be done by filtering the original dataset and only keeping the rows where the FIPS value is between 01001 (corresponding to Autauga, Alabama) and 56045 (corresponding to Weston, Wyoming).

In [ ]:
county_info = geo_info[geo_info['FIPS'].between(1001, 56045)]
county_info.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Population
1173,84001001,US,USA,840.0,1001.0,Autauga,Alabama,US,32.539527,-86.644082,"Autauga, Alabama, US",55869.0
1174,84001003,US,USA,840.0,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,"Baldwin, Alabama, US",223234.0
1175,84001005,US,USA,840.0,1005.0,Barbour,Alabama,US,31.868263,-85.387129,"Barbour, Alabama, US",24686.0
1176,84001007,US,USA,840.0,1007.0,Bibb,Alabama,US,32.996421,-87.125115,"Bibb, Alabama, US",22394.0
1177,84001009,US,USA,840.0,1009.0,Blount,Alabama,US,33.982109,-86.567906,"Blount, Alabama, US",57826.0


We re-format column and remove columns which we do not need.

In [ ]:
county_info = county_info[['FIPS','Admin2','Province_State','Lat','Long_','Population']]
county_info.rename(columns = {'FIPS':'fips', 'Province_State':'state', 'Admin2':'county',
                      'Lat':'lat', 'Long_':'long', 'Population':'population'}, inplace = True)
county_info.head()

,fips,county,state,lat,long,population
1173,1001.0,Autauga,Alabama,32.539527,-86.644082,55869.0
1174,1003.0,Baldwin,Alabama,30.727750,-87.722071,223234.0
1175,1005.0,Barbour,Alabama,31.868263,-85.387129,24686.0
1176,1007.0,Bibb,Alabama,32.996421,-87.125115,22394.0
1177,1009.0,Blount,Alabama,33.982109,-86.567906,57826.0


Change the FIPS value to be 5-character strings.

In [ ]:
county_info.fips = (county_info.fips.astype(int)).astype(object)
county_info['fips'] = county_info['fips'].apply('{:0>5}'.format)
county_info.head()

,fips,county,state,lat,long,population
1173,01001,Autauga,Alabama,32.539527,-86.644082,55869.0
1174,01003,Baldwin,Alabama,30.727750,-87.722071,223234.0
1175,01005,Barbour,Alabama,31.868263,-85.387129,24686.0
1176,01007,Bibb,Alabama,32.996421,-87.125115,22394.0
1177,01009,Blount,Alabama,33.982109,-86.567906,57826.0


Remove all Alaska and Hawaii counties from the dataset as we are only working with continental US data.

In [ ]:
county_info = county_info[(county_info.state != "Alaska") & (county_info.state != "Hawaii")]
county_info.reset_index(inplace=True, drop = True)
county_info.head()

,fips,county,state,lat,long,population
0,01001,Autauga,Alabama,32.539527,-86.644082,55869.0
1,01003,Baldwin,Alabama,30.727750,-87.722071,223234.0
2,01005,Barbour,Alabama,31.868263,-85.387129,24686.0
3,01007,Bibb,Alabama,32.996421,-87.125115,22394.0
4,01009,Blount,Alabama,33.982109,-86.567906,57826.0


Quick check to make sure all states in state_info appear in county_info and vice-versa (no extra states/counties).

In [ ]:
state_info['state'].unique() == county_info['state'].unique()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

## 1.3 Export State and County General Data


In [ ]:
state_info.to_csv('state_info.csv', index=False)
county_info.to_csv('county_info.csv', index=False)

# 2 COVID Data

In this section, we get COVID case data from the Johns Hopkins and NY Times GitHub pages. This section was done in preparation for writing our own functions to extract the data automatically for our dashboard. The final dashboard makes use of the covid_data.py file provided with our final submission.

As mentioned, we will be getting our COVID data from the Johns Hopkins GitHub database. In this database, each day has its own dedicated csv file named in the mm-dd-yyy format.In order to access the data, we will first generate a list of dates that we want to collect data for and that can later be incorporated into the URL we will be reading from.

Note: A week of data from 2020 will be included in the data set as we will later on be converting cumulative cases into active cases.

In [6]:
dates = pd.date_range(date(2020,12,22),date(2022,1,1)-timedelta(days=1),freq='d').strftime('%m-%d-%Y')
dates

Index(['12-22-2020', '12-23-2020', '12-24-2020', '12-25-2020', '12-26-2020',
       '12-27-2020', '12-28-2020', '12-29-2020', '12-30-2020', '12-31-2020',
       ...
       '12-22-2021', '12-23-2021', '12-24-2021', '12-25-2021', '12-26-2021',
       '12-27-2021', '12-28-2021', '12-29-2021', '12-30-2021', '12-31-2021'],
      dtype='object', length=375)

## 2.1 Get COVID data

In this section, we get data on confirmed cases from the Johns Hopkings GitHub repository by accessing and storing data for each day in 2021.

Note: To skip running this code (and avoid waiting), run the first cell in Section 2.2, which will get the data gathered in Section 2.1 from our own GitHub repository.

### 2.1.1 DataFrame Set Up

To store the data, we will start with the base county_info dataframe. Then, we will individually load the data file for each date, extract the data that we need, and add it to the county_info data frame. In the end, county_info will contain all the county-specific demographic/geo data, as well as all the covid data.

Note: Johns Hopkins updates their COVID data base daily, sometimes including the FIPS file used in the first section. For that reason, we will be loading the county_info data from the final file generated in the previous section and stored in our own GitHub repository.

In [3]:
url = 'https://raw.githubusercontent.com/AnjaDeric/MDA-TeamCroatia/main/Data/county_info.csv'
county_info = pd.read_csv(url)
# reformat FIPS into 5-character string
county_info['fips'] = county_info['fips'].apply('{:0>5}'.format)

In [4]:
covid_data = county_info.copy()
covid_data.head()

,fips,county,state,lat,long,population
0,01001,Autauga,Alabama,32.539527,-86.644082,55869
1,01003,Baldwin,Alabama,30.727750,-87.722071,223234
2,01005,Barbour,Alabama,31.868263,-85.387129,24686
3,01007,Bibb,Alabama,32.996421,-87.125115,22394
4,01009,Blount,Alabama,33.982109,-86.567906,57826


In [5]:
# get a list of all unique counties we will need data for
county_fips = covid_data['fips'].unique()

### 2.1.2 Get Data from Johns Hopkins GitHub

We then loop through all the necessary dates and collect the confirmed case data. Note: in the final iteration of the covid_data.py script, we use the time series file instead of accessing individual files for each date.

In [7]:
base_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/"

for date in dates:
  # read in file for each date
  url = base_url + date + ".csv"
  c2021 = pd.read_csv(url)

  # fill in missing FIPS with 0s and format to 5 digits
  c2021['FIPS'] = c2021['FIPS'].fillna(0)
  c2021.FIPS = (c2021.FIPS.astype(int)).astype(object)
  c2021['FIPS'] = c2021['FIPS'].apply('{:0>5}'.format)
  c2021.rename(columns = {'FIPS':'fips'}, inplace = True)

  # only keep rows were FIPS matches one of the US county FIPS
  c2021_counties = c2021[c2021['fips'].isin(county_fips)]

  # merge case data with the base dataframe containing county information
  covid_data = pd.merge(covid_data, c2021_counties[['fips','Confirmed']], on="fips", how="left")

  # reformat date
  date_col_name = 'd' + date.split('-')[0] + date.split('-')[1] + date.split('-')[2];
  covid_data.rename(columns = {'Confirmed':date_col_name}, inplace = True)


In [9]:
covid_data.head()

,fips,county,state,lat,long,population,d12222020,d12232020,d12242020,d12252020,...,d12222021,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021
0,01001,Autauga,Alabama,32.539527,-86.644082,55869,3841.0,3889.0,3942.0,3990.0,...,10711.0,10746.0,10758.0,10785.0,10789.0,10800.0,10828.0,10886.0,10953.0,11018.0
1,01003,Baldwin,Alabama,30.727750,-87.722071,223234,12155.0,12321.0,12521.0,12666.0,...,38628.0,38690.0,38771.0,38864.0,38878.0,38942.0,39084.0,39298.0,39583.0,39911.0
2,01005,Barbour,Alabama,31.868263,-85.387129,24686,1363.0,1383.0,1390.0,1396.0,...,3726.0,3730.0,3735.0,3740.0,3742.0,3742.0,3750.0,3777.0,3819.0,3860.0
3,01007,Bibb,Alabama,32.996421,-87.125115,22394,1660.0,1683.0,1711.0,1725.0,...,4409.0,4410.0,4410.0,4414.0,4420.0,4426.0,4452.0,4472.0,4514.0,4533.0
4,01009,Blount,Alabama,33.982109,-86.567906,57826,4313.0,4367.0,4405.0,4441.0,...,11002.0,11031.0,11041.0,11053.0,11064.0,11072.0,11096.0,11128.0,11193.0,11256.0


In [8]:
covid_data.to_csv('covid_data_raw_nts.csv', index=False)

## 2.2 Cleaning COVID Data

Load in the previously-saved raw COVID data.

In [12]:
# run to load saved data so you don't have to wait
covid_data = pd.read_csv('https://raw.githubusercontent.com/AnjaDeric/MDA-TeamCroatia/main/Data/Mid-Points/covid_data_raw_nts.csv')
covid_data['fips'] = covid_data['fips'].apply('{:0>5}'.format)
covid_data.head()

,fips,county,state,lat,long,population,d12222020,d12232020,d12242020,d12252020,...,d12222021,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021
0,01001,Autauga,Alabama,32.539527,-86.644082,55869,3841.0,3889.0,3942.0,3990.0,...,10711.0,10746.0,10758.0,10785.0,10789.0,10800.0,10828.0,10886.0,10953.0,11018.0
1,01003,Baldwin,Alabama,30.727750,-87.722071,223234,12155.0,12321.0,12521.0,12666.0,...,38628.0,38690.0,38771.0,38864.0,38878.0,38942.0,39084.0,39298.0,39583.0,39911.0
2,01005,Barbour,Alabama,31.868263,-85.387129,24686,1363.0,1383.0,1390.0,1396.0,...,3726.0,3730.0,3735.0,3740.0,3742.0,3742.0,3750.0,3777.0,3819.0,3860.0
3,01007,Bibb,Alabama,32.996421,-87.125115,22394,1660.0,1683.0,1711.0,1725.0,...,4409.0,4410.0,4410.0,4414.0,4420.0,4426.0,4452.0,4472.0,4514.0,4533.0
4,01009,Blount,Alabama,33.982109,-86.567906,57826,4313.0,4367.0,4405.0,4441.0,...,11002.0,11031.0,11041.0,11053.0,11064.0,11072.0,11096.0,11128.0,11193.0,11256.0


### 2.2.1 Missing Values

After gathering all the confirmed cases, we take a look at any rows with missing values. The only columns with missing values are the covid case count columns. 22 rows in total are missing all case count data.

In [13]:
covid_data[covid_data.isnull().any(axis=1)]

,fips,county,state,lat,long,population,d12222020,d12232020,d12242020,d12252020,...,d12222021,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021
2741,49001,Beaver,Utah,38.356571,-113.234223,6710,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2742,49003,Box Elder,Utah,41.521068,-113.083282,56046,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2743,49005,Cache,Utah,41.723306,-111.744367,128289,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2744,49007,Carbon,Utah,39.648348,-110.587251,20463,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2745,49009,Daggett,Utah,40.887983,-109.512109,950,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2747,49013,Duchesne,Utah,40.297723,-110.425237,19938,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2748,49015,Emery,Utah,38.996171,-110.701396,10012,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2749,49017,Garfield,Utah,37.854472,-111.441876,5051,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2750,49019,Grand,Utah,38.981038,-109.570449,9754,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2751,49021,Iron,Utah,37.859036,-113.289738,54839,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


These rows are all cases from Utah counties. Only around 7 counties for Utah actually have COVID data reported in this data set. To get data for the rest, we will be looking at a data set from another source (NY Times). For the time being, we drop the rows containing these Utah counties from the master covid_data file and work on it in a separate data frame. We first get a list of all Utah county FIPS that are missing data.



In [14]:
# extract all rows with missing data
missing_data = covid_data[covid_data.isnull().any(axis=1)]

# get a list of all FIPS with missing data
missing_fips = missing_data['fips'].unique()
missing_fips

array(['49001', '49003', '49005', '49007', '49009', '49013', '49015',
       '49017', '49019', '49021', '49023', '49025', '49027', '49029',
       '49031', '49033', '49039', '49041', '49047', '49053', '49055',
       '49057'], dtype=object)

In [15]:
# drop these rows from master file as we will be working on them separately
covid_data.drop(covid_data.loc[covid_data.isnull().any(axis=1)].index, inplace=True)

Now, we load the data from NY Times. Note: NY Times data is not used from the start since there are more missing values and cases of city data being reported separately from county data as compared to the Johns Hopkins dataset (overall higher need for data cleaning with the NY Times data).

In [16]:
# get 2021 NY Times data
nytimes_21 = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2021.csv")
nytimes_21.head()

,date,county,state,fips,cases,deaths
0,2021-01-01,Autauga,Alabama,1001.0,4239,50.0
1,2021-01-01,Baldwin,Alabama,1003.0,13823,169.0
2,2021-01-01,Barbour,Alabama,1005.0,1517,33.0
3,2021-01-01,Bibb,Alabama,1007.0,1854,46.0
4,2021-01-01,Blount,Alabama,1009.0,4693,63.0


In [17]:
# get 2020 NY Times data
nytimes_20 = pd.read_csv("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties-2020.csv")
nytimes_20.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


Create a function to extract the data for the missing counties.



In [18]:
def extract_times_data(nytimes):
  # reformat FIPS to match 5-digit format
  nytimes['fips'] = nytimes['fips'].fillna(0)
  nytimes.fips = (nytimes.fips.astype(int)).astype(object)
  nytimes['fips'] = nytimes['fips'].apply('{:0>5}'.format)

  # get only rows with missing Utah counties
  nytimes = nytimes[nytimes['fips'].isin(missing_fips)]

  # change date to format used for covid_data columns
  nytimes[['yy','mm','dd']]=nytimes['date'].str.split('-', expand=True)
  nytimes["date"] = "d" + nytimes["mm"] + nytimes["dd"] + nytimes["yy"]

  # transform into wide table and add FIPS code
  nytimes_wide = nytimes.pivot(index='county',columns='date',values='cases')
  nytimes_wide['fips'] = nytimes['fips'].unique()

  return nytimes_wide

In [19]:
# extract 2021 data
nytimes_wide_21 = extract_times_data(nytimes_21)
nytimes_wide_21.head()

date,d01012021,d01022021,d01032021,d01042021,d01052021,d01062021,d01072021,d01082021,d01092021,d01102021,...,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021,fips
county,,,,,,,,,,,,,,,,,,,,,
Beaver,405,410,410,414,424,427,436,439,445,445,...,1231,1231,1231,1231,1232,1237,1240,1240,1243,49001
Box Elder,3571,3658,3704,3733,3778,3872,3945,3997,4026,4053,...,10283,10283,10283,10283,10329,10339,10364,10364,10393,49003
Cache,11167,11351,11417,11500,11620,11807,11941,12065,12144,12211,...,24702,24702,24702,24702,24896,24943,25077,25077,25185,49005
Carbon,1072,1075,1079,1096,1101,1113,1127,1138,1147,1147,...,3712,3713,3713,3713,3713,3719,3728,3735,3749,49007
Daggett,18,18,18,18,20,21,22,22,22,22,...,120,120,120,120,119,122,123,123,123,49009


In [20]:
# extract 2020 data
nytimes_wide_20 = extract_times_data(nytimes_20)
nytimes_wide_20.drop(nytimes_wide_20.columns[0:len(nytimes_wide_20.columns)-11], axis=1, inplace=True)
nytimes_wide_20.head()


date,d12222020,d12232020,d12242020,d12252020,d12262020,d12272020,d12282020,d12292020,d12302020,d12312020,fips
county,,,,,,,,,,,
Beaver,312.0,332.0,341.0,341.0,355.0,355.0,362.0,375.0,395.0,405.0,49053
Box Elder,3225.0,3286.0,3340.0,3340.0,3381.0,3402.0,3428.0,3464.0,3519.0,3571.0,49057
Cache,10355.0,10454.0,10539.0,10539.0,10700.0,10723.0,10790.0,10887.0,10967.0,11167.0,49003
Carbon,1011.0,1019.0,1021.0,1021.0,1028.0,1031.0,1048.0,1056.0,1064.0,1072.0,49005
Daggett,13.0,13.0,15.0,15.0,15.0,15.0,17.0,17.0,17.0,18.0,49021


Combine the 2021 and 2020 data into one dataframe with all missing values filled in.

In [21]:
# start with dataframe with only county info
missing_data_base = missing_data[['fips','county','state','lat','long','population']]
# merge with 2020 and 2021 data
missing_data_all = pd.merge(missing_data_base, nytimes_wide_20, on='fips', how="left")
missing_data_all = pd.merge(missing_data_all, nytimes_wide_21, on='fips', how="left")
missing_data_all.head()

,fips,county,state,lat,long,population,d12222020,d12232020,d12242020,d12252020,...,d12222021,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021
0,49001,Beaver,Utah,38.356571,-113.234223,6710,13415.0,13622.0,13797.0,13797.0,...,1231,1231,1231,1231,1231,1232,1237,1240,1240,1243
1,49003,Box Elder,Utah,41.521068,-113.083282,56046,10355.0,10454.0,10539.0,10539.0,...,10271,10283,10283,10283,10283,10329,10339,10364,10364,10393
2,49005,Cache,Utah,41.723306,-111.744367,128289,1011.0,1019.0,1021.0,1021.0,...,24658,24702,24702,24702,24702,24896,24943,25077,25077,25185
3,49007,Carbon,Utah,39.648348,-110.587251,20463,2912.0,2948.0,2964.0,2964.0,...,3708,3712,3713,3713,3713,3713,3719,3728,3735,3749
4,49009,Daggett,Utah,40.887983,-109.512109,950,17602.0,17783.0,18124.0,18124.0,...,118,120,120,120,120,119,122,123,123,123


Finally, add the Utah county data to the master data frame for covid cases.

In [22]:
covid_data_final = pd.concat([covid_data,missing_data_all],ignore_index=True)
covid_data_final

,fips,county,state,lat,long,population,d12222020,d12232020,d12242020,d12252020,...,d12222021,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021
0,01001,Autauga,Alabama,32.539527,-86.644082,55869,3841.0,3889.0,3942.0,3990.0,...,10711.0,10746.0,10758.0,10785.0,10789.0,10800.0,10828.0,10886.0,10953.0,11018.0
1,01003,Baldwin,Alabama,30.727750,-87.722071,223234,12155.0,12321.0,12521.0,12666.0,...,38628.0,38690.0,38771.0,38864.0,38878.0,38942.0,39084.0,39298.0,39583.0,39911.0
2,01005,Barbour,Alabama,31.868263,-85.387129,24686,1363.0,1383.0,1390.0,1396.0,...,3726.0,3730.0,3735.0,3740.0,3742.0,3742.0,3750.0,3777.0,3819.0,3860.0
3,01007,Bibb,Alabama,32.996421,-87.125115,22394,1660.0,1683.0,1711.0,1725.0,...,4409.0,4410.0,4410.0,4414.0,4420.0,4426.0,4452.0,4472.0,4514.0,4533.0
4,01009,Blount,Alabama,33.982109,-86.567906,57826,4313.0,4367.0,4405.0,4441.0,...,11002.0,11031.0,11041.0,11053.0,11064.0,11072.0,11096.0,11128.0,11193.0,11256.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3101,49041,Sevier,Utah,38.748371,-111.805027,21620,59.0,63.0,63.0,63.0,...,4086.0,4094.0,4094.0,4094.0,4094.0,4128.0,4142.0,4160.0,4160.0,4172.0
3102,49047,Uintah,Utah,40.124915,-109.517442,35734,517.0,523.0,527.0,527.0,...,5663.0,5678.0,5678.0,5678.0,5678.0,5693.0,5697.0,5710.0,5710.0,5721.0
3103,49053,Washington,Utah,37.280035,-113.504698,177556,312.0,332.0,341.0,341.0,...,32240.0,32303.0,32303.0,32303.0,32303.0,32560.0,32639.0,32767.0,32767.0,32888.0
3104,49055,Wayne,Utah,38.323358,-110.909680,2711,65.0,65.0,66.0,66.0,...,298.0,301.0,301.0,301.0,301.0,302.0,302.0,303.0,303.0,304.0


We check one final time to see if there are any rows with any missing data.

In [23]:
covid_data_final[covid_data_final.isnull().any(axis=1)]

,fips,county,state,lat,long,population,d12222020,d12232020,d12242020,d12252020,...,d12222021,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021


Since there are no more missing cells, we save the final csv file.

In [24]:
covid_data_final.to_csv('covid_data_clean.csv', index=False)

## 2.3 Getting Active Case Counts

In the current data set (created in Section 2.2), the case counts listed are cumulative. For instance, in Autauga, AL, there are 4268 cumulative cases on January 1st, and 4305 cases on January 2nd. This means that on January second, 38 new cases were detected, on top of the already active cases. A case of COVID can be considered active 10 days after its detection (source: WHO). 

In order to give the best travel recommendations, we want to work with active cases, as opposed to cumulative or newly detected cases. The following section converts the cumulative case counts into active case counts. 

In [25]:
# run to load saved data so you don't have to wait for previous steps
covid_data = pd.read_csv('https://raw.githubusercontent.com/AnjaDeric/MDA-TeamCroatia/main/Data/Mid-Points/covid_data_clean.csv')
covid_data['fips'] = covid_data['fips'].apply('{:0>5}'.format)
covid_data.head()

,fips,county,state,lat,long,population,d12222020,d12232020,d12242020,d12252020,...,d12222021,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021
0,01001,Autauga,Alabama,32.539527,-86.644082,55869,3841.0,3889.0,3942.0,3990.0,...,10711.0,10746.0,10758.0,10785.0,10789.0,10800.0,10828.0,10886.0,10953.0,11018.0
1,01003,Baldwin,Alabama,30.727750,-87.722071,223234,12155.0,12321.0,12521.0,12666.0,...,38628.0,38690.0,38771.0,38864.0,38878.0,38942.0,39084.0,39298.0,39583.0,39911.0
2,01005,Barbour,Alabama,31.868263,-85.387129,24686,1363.0,1383.0,1390.0,1396.0,...,3726.0,3730.0,3735.0,3740.0,3742.0,3742.0,3750.0,3777.0,3819.0,3860.0
3,01007,Bibb,Alabama,32.996421,-87.125115,22394,1660.0,1683.0,1711.0,1725.0,...,4409.0,4410.0,4410.0,4414.0,4420.0,4426.0,4452.0,4472.0,4514.0,4533.0
4,01009,Blount,Alabama,33.982109,-86.567906,57826,4313.0,4367.0,4405.0,4441.0,...,11002.0,11031.0,11041.0,11053.0,11064.0,11072.0,11096.0,11128.0,11193.0,11256.0


### 2.3.1 Getting Newly Detected Cases

To get newly detected cases, we have to subtract the cumulative case count for each day from the previous cumulative case count. 

In [26]:
new_cases = covid_data.copy()
# subtract date columns consecutively
new_cases.loc[:,'d12222020':'d12312021'] = new_cases.loc[:,'d12222020':'d12312021'].diff(axis=1)
# drop the first date (december 22)
new_cases.drop(['d12222020'], axis=1, inplace=True)
new_cases.head()

,fips,county,state,lat,long,population,d12232020,d12242020,d12252020,d12262020,...,d12222021,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021
0,01001,Autauga,Alabama,32.539527,-86.644082,55869,48.0,53.0,48.0,9.0,...,19.0,35.0,12.0,27.0,4.0,11.0,28.0,58.0,67.0,65.0
1,01003,Baldwin,Alabama,30.727750,-87.722071,223234,166.0,200.0,145.0,42.0,...,59.0,62.0,81.0,93.0,14.0,64.0,142.0,214.0,285.0,328.0
2,01005,Barbour,Alabama,31.868263,-85.387129,24686,20.0,7.0,6.0,2.0,...,1.0,4.0,5.0,5.0,2.0,0.0,8.0,27.0,42.0,41.0
3,01007,Bibb,Alabama,32.996421,-87.125115,22394,23.0,28.0,14.0,14.0,...,3.0,1.0,0.0,4.0,6.0,6.0,26.0,20.0,42.0,19.0
4,01009,Blount,Alabama,33.982109,-86.567906,57826,54.0,38.0,36.0,5.0,...,12.0,29.0,10.0,12.0,11.0,8.0,24.0,32.0,65.0,63.0


In some cases where there are errors in data reporting, the cumulative case count decreases between two days, resulting in a negative new case count. This is an error in data reporting, so we mitigate it by just re-setting the new case count to 0.

In [27]:
# set all negative values to 0
case_cols = (new_cases.columns[6:]).tolist()
new_cases[case_cols] = new_cases[case_cols].mask(new_cases[case_cols] < 0, 0)
new_cases.head()

,fips,county,state,lat,long,population,d12232020,d12242020,d12252020,d12262020,...,d12222021,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021
0,01001,Autauga,Alabama,32.539527,-86.644082,55869,48.0,53.0,48.0,9.0,...,19.0,35.0,12.0,27.0,4.0,11.0,28.0,58.0,67.0,65.0
1,01003,Baldwin,Alabama,30.727750,-87.722071,223234,166.0,200.0,145.0,42.0,...,59.0,62.0,81.0,93.0,14.0,64.0,142.0,214.0,285.0,328.0
2,01005,Barbour,Alabama,31.868263,-85.387129,24686,20.0,7.0,6.0,2.0,...,1.0,4.0,5.0,5.0,2.0,0.0,8.0,27.0,42.0,41.0
3,01007,Bibb,Alabama,32.996421,-87.125115,22394,23.0,28.0,14.0,14.0,...,3.0,1.0,0.0,4.0,6.0,6.0,26.0,20.0,42.0,19.0
4,01009,Blount,Alabama,33.982109,-86.567906,57826,54.0,38.0,36.0,5.0,...,12.0,29.0,10.0,12.0,11.0,8.0,24.0,32.0,65.0,63.0


### 2.3.2 Getting Active Case Counts

Next, we convert new case counts to active case counts. We considerer a new COVID case active for 10 days, so we take a rolling sum over 10 days.

In [28]:
active_cases = new_cases.copy()
# take a rolling sum over 10 days
active_cases.loc[:,'d12232020':'d12312021'] = active_cases.loc[:,'d12232020':'d12312021'].rolling(10,axis=1).sum()
# remove the 2020 dates
active_cases.drop(['d12232020','d12242020','d12252020','d12262020','d12272020','d12282020','d12292020','d12302020','d12312020'], axis=1, inplace=True)
active_cases.head()

,fips,county,state,lat,long,population,d01012021,d01022021,d01032021,d01042021,...,d12222021,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021
0,01001,Autauga,Alabama,32.539527,-86.644082,55869,398.0,379.0,363.0,346.0,...,112.0,141.0,121.0,144.0,143.0,139.0,165.0,217.0,274.0,326.0
1,01003,Baldwin,Alabama,30.727750,-87.722071,223234,1668.0,1634.0,1543.0,1521.0,...,314.0,369.0,373.0,451.0,437.0,485.0,600.0,792.0,1062.0,1342.0
2,01005,Barbour,Alabama,31.868263,-85.387129,24686,154.0,145.0,140.0,137.0,...,10.0,14.0,17.0,21.0,22.0,20.0,28.0,54.0,94.0,135.0
3,01007,Bibb,Alabama,32.996421,-87.125115,22394,194.0,180.0,171.0,160.0,...,30.0,31.0,25.0,26.0,29.0,33.0,56.0,75.0,114.0,127.0
4,01009,Blount,Alabama,33.982109,-86.567906,57826,380.0,362.0,341.0,330.0,...,142.0,167.0,163.0,149.0,145.0,131.0,142.0,169.0,224.0,266.0


We save the final active case file for 2021.

In [ ]:
active_cases.to_csv('active_cases_2021.csv', index=False)

## 2.4 Scaling By Population


The active case counts provided are total case counts, regardless of the population. In order to more accurately represent the incidence of COVID cases, we scale the case counts by population (per 100,000 people)

In [29]:
# run to load saved data so you don't have to wait for previous steps
active_cases = pd.read_csv('https://raw.githubusercontent.com/AnjaDeric/MDA-TeamCroatia/main/Data/active_cases_2021.csv')
active_cases['fips'] = active_cases['fips'].apply('{:0>5}'.format)
active_cases.head()

,fips,county,state,lat,long,population,d01012021,d01022021,d01032021,d01042021,...,d12222021,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021
0,01001,Autauga,Alabama,32.539527,-86.644082,55869,398.0,379.0,363.0,346.0,...,112.0,141.0,121.0,144.0,143.0,139.0,165.0,217.0,274.0,326.0
1,01003,Baldwin,Alabama,30.727750,-87.722071,223234,1668.0,1634.0,1543.0,1521.0,...,314.0,369.0,373.0,451.0,437.0,485.0,600.0,792.0,1062.0,1342.0
2,01005,Barbour,Alabama,31.868263,-85.387129,24686,154.0,145.0,140.0,137.0,...,10.0,14.0,17.0,21.0,22.0,20.0,28.0,54.0,94.0,135.0
3,01007,Bibb,Alabama,32.996421,-87.125115,22394,194.0,180.0,171.0,160.0,...,30.0,31.0,25.0,26.0,29.0,33.0,56.0,75.0,114.0,127.0
4,01009,Blount,Alabama,33.982109,-86.567906,57826,380.0,362.0,341.0,330.0,...,142.0,167.0,163.0,149.0,145.0,131.0,142.0,169.0,224.0,266.0


We divide all case columns by the respective county population and multiply by 100,000 to get active cases scaled by population (with units of per 100,000 people)

In [30]:
active_cases.iloc[:,6:] = active_cases.iloc[:,6:].div(active_cases.population, axis=0)*100000
active_cases.head()

,fips,county,state,lat,long,population,d01012021,d01022021,d01032021,d01042021,...,d12222021,d12232021,d12242021,d12252021,d12262021,d12272021,d12282021,d12292021,d12302021,d12312021
0,01001,Autauga,Alabama,32.539527,-86.644082,55869,712.380748,678.372622,649.734200,619.305876,...,200.468954,252.376094,216.578067,257.745798,255.955897,248.796291,295.333727,388.408599,490.432977,583.507849
1,01003,Baldwin,Alabama,30.727750,-87.722071,223234,747.198007,731.967353,691.202953,681.347823,...,140.659577,165.297401,167.089243,202.030157,195.758711,217.260812,268.776262,354.784665,475.733983,601.162905
2,01005,Barbour,Alabama,31.868263,-85.387129,24686,623.835372,587.377461,567.123066,554.970429,...,40.508790,56.712307,68.864944,85.068460,89.119339,81.017581,113.424613,218.747468,380.782630,546.868671
3,01007,Bibb,Alabama,32.996421,-87.125115,22394,866.303474,803.786729,763.597392,714.477092,...,133.964455,138.429937,111.637046,116.102527,129.498973,147.360900,250.066982,334.911137,509.064928,567.116192
4,01009,Blount,Alabama,33.982109,-86.567906,57826,657.143845,626.015979,589.700135,570.677550,...,245.564279,288.797427,281.880123,257.669560,250.752257,226.541694,245.564279,292.256079,387.369004,460.000692


For the map that we are working with in our dashboard, the FIPS values are based on data prior to 2015. There is one county in South Dakota which changed it's name and FIPS value in 2015. We re-set its FIPS value to the pre-2015 FIPS in order to account for this.

In [31]:
active_cases['fips'].replace({'46102': '46113'}, inplace=True)

Finally, we save the scaled active case data.

In [ ]:
active_cases.to_csv('active_cases_2021_scaled.csv', index=False)

# 3 County Adjacency

In this section, we create a dataframe that lists pairs of counties that border each other in the US. This will be used to create relationships between the counties in our graph model.

## 3.1 Load County Adjacency Data

Start by loading in original data from the US Census Bureau ([link](https://www.census.gov/geographies/reference-files/2010/geo/county-adjacency.html)). Note: the text data file from the website was downloaded directly and saved as a CSV file (with custom delimiter option); no additional data processing was done in Excel.

In [ ]:
url = "https://raw.githubusercontent.com/AnjaDeric/MDA-TeamCroatia/main/Data/Mid-Points/adjacent_counties.csv"
adj = pd.read_csv(url,encoding= 'unicode_escape')
adj.head()

,Column1,Column2,Column3,Column4
0,"Autauga County, AL",01001,"Autauga County, AL",1001.0
1,NaN,"Chilton County, AL",01021,NaN
2,NaN,"Dallas County, AL",01047,NaN
3,NaN,"Elmore County, AL",01051,NaN
4,NaN,"Lowndes County, AL",01085,NaN


## 3.2 Clean County Adjacency Data

Move columns 2 and 3 over to the right in rows following each new county.

In [ ]:
adj.loc[adj['Column1'].isnull(),'Column4'] = adj['Column3']
adj.loc[adj['Column1'].isnull(),'Column3'] = adj['Column2']
adj.loc[adj['Column1'].isnull(),'Column2'] = np.nan
adj.head()

,Column1,Column2,Column3,Column4
0,"Autauga County, AL",01001,"Autauga County, AL",1001.0
1,NaN,NaN,"Chilton County, AL",01021
2,NaN,NaN,"Dallas County, AL",01047
3,NaN,NaN,"Elmore County, AL",01051
4,NaN,NaN,"Lowndes County, AL",01085


After manually checking the new dataframe, there is an error in one row, where Column4 contains a bordering county name and Column3 contains a fips code. Correct this mistake.

In [ ]:
adj.loc[adj['Column4']=="Blue Earth County, MN",'Column3'] = "Blue Earth County, MN"
adj.loc[adj['Column4']=="Blue Earth County, MN",'Column4'] = "27013"

Convert fips code columns to object data type and add leading zeros for counties that are missing them. Aditionally, adjust column names to be accurate.

In [ ]:
adj.Column4 = (adj.Column4.astype(int)).astype(object)
adj['Column4'] = adj['Column4'].apply('{:0>5}'.format)
adj.rename(columns = {'Column1':'county', 'Column2':'county_fips',
                      'Column3':'bcounty', 'Column4':'bcounty_fips'}, inplace = True)
adj.head()

,county,county_fips,bcounty,bcounty_fips
0,"Autauga County, AL",01001,"Autauga County, AL",01001
1,NaN,NaN,"Chilton County, AL",01021
2,NaN,NaN,"Dallas County, AL",01047
3,NaN,NaN,"Elmore County, AL",01051
4,NaN,NaN,"Lowndes County, AL",01085


For the first 2 columns, fill the missing values (NaN) with the countty and code in the row above.

In [ ]:
adj['county'] = adj['county'].mask(adj['county'].eq('')).ffill()
adj['county_fips'] = adj['county_fips'].mask(adj['county_fips'].eq('')).ffill()
adj.head()

,county,county_fips,bcounty,bcounty_fips
0,"Autauga County, AL",01001,"Autauga County, AL",01001
1,"Autauga County, AL",01001,"Chilton County, AL",01021
2,"Autauga County, AL",01001,"Dallas County, AL",01047
3,"Autauga County, AL",01001,"Elmore County, AL",01051
4,"Autauga County, AL",01001,"Lowndes County, AL",01085


Remove rows where a county is bordering itself.

In [ ]:
adj = adj[adj['county_fips'] != adj['bcounty_fips']]
adj.head()

,county,county_fips,bcounty,bcounty_fips
1,"Autauga County, AL",01001,"Chilton County, AL",01021
2,"Autauga County, AL",01001,"Dallas County, AL",01047
3,"Autauga County, AL",01001,"Elmore County, AL",01051
4,"Autauga County, AL",01001,"Lowndes County, AL",01085
5,"Autauga County, AL",01001,"Montgomery County, AL",01101


Split county names and states into separate columns.

In [ ]:
adj[['county_name', 'county_state']] = adj['county'].str.split(',', expand=True)
adj[['bcounty_name', 'bcounty_state']] = adj['bcounty'].str.split(',', expand=True)
adj = adj[['county_name','county_state','county_fips','bcounty_name','bcounty_state','bcounty_fips']]
adj.head()

,county_name,county_state,county_fips,bcounty_name,bcounty_state,bcounty_fips
1,Autauga County,AL,01001,Chilton County,AL,01021
2,Autauga County,AL,01001,Dallas County,AL,01047
3,Autauga County,AL,01001,Elmore County,AL,01051
4,Autauga County,AL,01001,Lowndes County,AL,01085
5,Autauga County,AL,01001,Montgomery County,AL,01101


Drop any rows where either of the counties is not on our list of continental US counties.

In [ ]:
# check to see county FIPS list
county_fips = active_cases['fips'].unique()

In [ ]:
# only keep rows were FIPS matches one of the US county FIPS in the list
adj = adj[adj['county_fips'].isin(county_fips)]
adj = adj[adj['bcounty_fips'].isin(county_fips)]

Drop duplicate rows where the same two bordering counties are being considered. This is done based on their combined FIPS codes sorted by value, so that any 2 rows that have the same 2 counties will have the same combined FIPS.

In [ ]:
adj['county_combined'] = [''.join(x) for x in np.sort(adj[['county_fips','bcounty_fips']], axis=1)]
adj_final = adj.drop_duplicates(subset=['county_combined'], keep='first')
#adj_final = adj_final.drop('county_combined', 1)
adj_final = adj_final.reset_index(drop=True)

In [ ]:
adj_final.head()

,county_name,county_state,county_fips,bcounty_name,bcounty_state,bcounty_fips,county_combined
0,Autauga County,AL,01001,Chilton County,AL,01021,0100101021
1,Autauga County,AL,01001,Dallas County,AL,01047,0100101047
2,Autauga County,AL,01001,Elmore County,AL,01051,0100101051
3,Autauga County,AL,01001,Lowndes County,AL,01085,0100101085
4,Autauga County,AL,01001,Montgomery County,AL,01101,0100101101


## 3.3 Complete and Export Data

The data set is now complete and corrected. The format is as follows:

*   **county_name**: name of the county
*   **county_state**: state the county is in 
*   **county_fips**: FIPS code of the county
*   **bcounty_name**: name of the bordering county
*   **bcounty_state**: state the bordering county is in 
*   **bcounty_fips**: FIPS code of the bordering county
*   **county_combined**: FIPS codes of the two counties combined into one line and sorted by value

Export the data set into a new csv file.



In [ ]:
adj_final.to_csv('adjacent_counties_corrected.csv', index=False)

# 4 County Distance Data

In this section, we add distance data to out county adjacency file. This will allow us to calculate the shortest path based on driving distances and durations. 

We start off by reading in the adjacent county information and formatting FIPS to 5-character strings.

In [ ]:
adj_final = pd.read_csv('https://raw.githubusercontent.com/AnjaDeric/MDA-TeamCroatia/main/Data/Mid-Points/adjacent_counties_corrected.csv')
adj_final['county_fips'] = adj_final['county_fips'].apply('{:0>5}'.format)
adj_final['bcounty_fips'] = adj_final['bcounty_fips'].apply('{:0>5}'.format)
adj_final['county_combined'] = adj_final['county_combined'].apply('{:0>10}'.format)
adj_final.head()

,county_name,county_state,county_fips,bcounty_name,bcounty_state,bcounty_fips,county_combined
0,Autauga County,AL,01001,Chilton County,AL,01021,0100101021
1,Autauga County,AL,01001,Dallas County,AL,01047,0100101047
2,Autauga County,AL,01001,Elmore County,AL,01051,0100101051
3,Autauga County,AL,01001,Lowndes County,AL,01085,0100101085
4,Autauga County,AL,01001,Montgomery County,AL,01101,0100101101


## 4.1 Great-Circle Distances

These inter-county distances were collected from the [National Bureau of Economic Research](https://www.nber.org/research/data/county-distance-database) (Section: Census County Distance Data, Year: 2010, Census County Distance: ∞). They are great-circle distances calculated using the Havershine formula and as a result, they might not match the actual driving distance between the counties, but they give a good overall estimate of their level of separation. 

Additionally, the raw data includes the distances between all combinations of counties, not just bordering counties. As a result, the file downloaded directly from the website is too large to be stored on GitHub. We initially loaded it in by uploading it to Google Drive, but will now use a filtered version which only contains the relevant counties and can thus be stored on GitHub.

To get the filtered version, we combined the FIPS codes of each pair of counties, and only kept and saved those combinations which are also found in the adj_final data set from Section 3: County Adjacency. The following piece of code was used to achieve that:

```
# Extract all bordering county FIPS combos
combo_fips = adj_final['county_combined'].unique()

# Filter great-circle distance file to only include relevant county combinations and save filtered file
dist = dist[dist['county_combined'].isin(combo_fips)]
dist.to_csv('GC_distances_filtered.csv', index=False)
```



We can now load and use the filtered GC_distances file.

In [ ]:
dist = pd.read_csv('https://raw.githubusercontent.com/AnjaDeric/MDA-TeamCroatia/main/Data/Mid-Points/GC_distances.csv')
dist['county1'] = dist['county1'].apply('{:0>5}'.format)
dist['county2'] = dist['county2'].apply('{:0>5}'.format)
dist.head()

,county1,mi_to_county,county2
0,01001,22.462994,01021
1,01001,26.844687,01085
2,01001,29.517585,01051
3,01001,30.776371,01047
4,01001,34.493443,01101


Generate a combined FIPS code and remove all duplicated entries.

In [ ]:
dist['county_combined'] = [''.join(x) for x in np.sort(dist[['county1','county2']], axis=1)]
dist = dist.drop_duplicates(subset=['county_combined'], keep='first')
dist.head()

,county1,mi_to_county,county2,county_combined
0,01001,22.462994,01021,0100101021
1,01001,26.844687,01085,0100101085
2,01001,29.517585,01051,0100101051
3,01001,30.776371,01047,0100101047
4,01001,34.493443,01101,0100101101


Merge the ajecent county file with the GC distances.

In [ ]:
adj_distances_GC = pd.merge(adj_final, dist[['mi_to_county','county_combined']], on="county_combined", how="left")
adj_distances_GC.head()

,county_name,county_state,county_fips,bcounty_name,bcounty_state,bcounty_fips,county_combined,mi_to_county
0,Autauga County,AL,01001,Chilton County,AL,01021,0100101021,22.462994
1,Autauga County,AL,01001,Dallas County,AL,01047,0100101047,30.776371
2,Autauga County,AL,01001,Elmore County,AL,01051,0100101051,29.517585
3,Autauga County,AL,01001,Lowndes County,AL,01085,0100101085,26.844687
4,Autauga County,AL,01001,Montgomery County,AL,01101,0100101101,34.493443


Calculate the distances in kilometers instead of miles

In [ ]:
adj_distances_GC['gc_dist_km'] = round(adj_distances_GC['mi_to_county']*1.60934,1)
adj_distances_GC = adj_distances_GC.drop(['mi_to_county'], axis=1)
adj_distances_GC.head()

,county_name,county_state,county_fips,bcounty_name,bcounty_state,bcounty_fips,county_combined,gc_dist_km
0,Autauga County,AL,01001,Chilton County,AL,01021,0100101021,36.2
1,Autauga County,AL,01001,Dallas County,AL,01047,0100101047,49.5
2,Autauga County,AL,01001,Elmore County,AL,01051,0100101051,47.5
3,Autauga County,AL,01001,Lowndes County,AL,01085,0100101085,43.2
4,Autauga County,AL,01001,Montgomery County,AL,01101,0100101101,55.5


Check one final time to confirm all bordering counties have a great-circle distance between them (no missing values in mi_to_county column).

In [ ]:
adj_distances_GC.isna().sum()

county_name        0
county_state       0
county_fips        0
bcounty_name       0
bcounty_state      0
bcounty_fips       0
county_combined    0
gc_dist_km         0
dtype: int64

In [ ]:
adj_distances_GC.to_csv('adj_distances_GC.csv', index=False)

## 4.2 Driving Distances

As mentioned in Section 4.1, great-circle distances do not completely accurately represent the driving distances between counties. To get actual driving distances (and driving duration), we will be using the [Distance Matrix API](https://distancematrix.ai/dev#travel_modes).

### 4.2.1 Prepare Final Data Frame

Make a copy of original adjacent county data frame to work with.

In [ ]:
adj_final_api = adj_distances_GC.copy()
adj_final_api.head()

,county_name,county_state,county_fips,bcounty_name,bcounty_state,bcounty_fips,county_combined,gc_dist_km
0,Autauga County,AL,01001,Chilton County,AL,01021,0100101021,36.2
1,Autauga County,AL,01001,Dallas County,AL,01047,0100101047,49.5
2,Autauga County,AL,01001,Elmore County,AL,01051,0100101051,47.5
3,Autauga County,AL,01001,Lowndes County,AL,01085,0100101085,43.2
4,Autauga County,AL,01001,Montgomery County,AL,01101,0100101101,55.5


Add origin and destination latitude and longitude to the data frame by merging it with the county_info data frame based on the FIPS code.

In [ ]:
def add_lat_long(adj_final_api,county_info,location):
  # select correct FIPS depending on if request is for origin or destination
  if(location=="origin"):
    left_col = "county_fips"
  else:
    left_col = "bcounty_fips"
  
  # merge latitude and longitude with the data frame
  adj_final_api = pd.merge(adj_final_api, county_info[['lat','long','fips']], 
                         left_on=left_col, right_on='fips', how="left")
  
  # add a column with origin/destination coordinate pairs (lat, long)
  adj_final_api[location] = adj_final_api.lat.astype(str) + ',' + adj_final_api.long.astype(str)

  # return df with coordinate pairs and all other variables dropped
  return adj_final_api.drop(['fips','lat','long'], axis=1)

In [ ]:
adj_final_api = add_lat_long(adj_final_api, county_info, 'origin')
adj_final_api = add_lat_long(adj_final_api, county_info, 'dest')

Add columns for distance and duration measurements to the data frame.

In [ ]:
adj_final_api['distance'] = np.nan
adj_final_api['duration'] = np.nan
adj_final_api.head()

,county_name,county_state,county_fips,bcounty_name,bcounty_state,bcounty_fips,county_combined,gc_dist_km,origin,dest,distance,duration
0,Autauga County,AL,01001,Chilton County,AL,01021,0100101021,36.2,"32.53952745,-86.64408227","32.85044126,-86.7173256",NaN,NaN
1,Autauga County,AL,01001,Dallas County,AL,01047,0100101047,49.5,"32.53952745,-86.64408227","32.32688101,-87.1086671",NaN,NaN
2,Autauga County,AL,01001,Elmore County,AL,01051,0100101051,47.5,"32.53952745,-86.64408227","32.59785413,-86.14415284",NaN,NaN
3,Autauga County,AL,01001,Lowndes County,AL,01085,0100101085,43.2,"32.53952745,-86.64408227","32.1597283,-86.65158371",NaN,NaN
4,Autauga County,AL,01001,Montgomery County,AL,01101,0100101101,55.5,"32.53952745,-86.64408227","32.2206831,-86.20969272",NaN,NaN


### 4.2.2 API Prep + Requests
To get the necessary data, we are using a free API key which allows for 5000 requests and has a trial period of 7 days. 2 API keys in total were used to get all the necessary data.

Note: As the free API key expires after 7 days, this section of the code will not run past 7 days after requesting the key or after exceeding the 5000 request limit. We still include this section and all of our code, but the trial and the key has already expired. Data collected from the API (raw version) is stored in our GitHub repository and is loaded in Section 4.2.3.

We first specify some key parameters for our API request, including the base URL, our key, as well as the mode of transportation and traffic model.

In [ ]:
BASE_URL = "https://api.distancematrix.ai"
api_key = "mM8V0cY63RO4kNquWS3C2X1ClUxlp"
mode = "driving"
traffic_model = "best_guess"

The following block of code generates the API request URLs and makes the actual requests. The request URLs are generated by combining the base URL with all the necessary parameters, including the latitude and longitude of the origin and the destination.

Once a response is received, the distance and the duration elements are extracted and stored in the final adjacency data frame.

In [ ]:
for idx in range(0,0):
  origin = adj_final_api.loc[idx, 'origin']
  destination = adj_final_api.loc[idx, 'dest']

  url = "{base_url}/maps/api/distancematrix/json" \
          "?key={api_key}" \
          "&origins={origin}" \
          "&destinations={destination}" \
          "&mode={mode}" \
          "&traffic_model={traffic_model}".format(base_url=BASE_URL,
                                                    api_key=api_key,
                                                    origin=origin,
                                                    destination=destination,
                                                    mode=mode,
                                                    traffic_model=traffic_model)
  # logging.info("URL: %s" % url)
  # result = requests.get(url)

  res_json = result.json()
  # if a driving path exists, extract the distance and duration
  if(res_json['rows'][0]['elements'][0]['status']=='OK'):
    adj_final_api.loc[idx, 'distance'] = res_json['rows'][0]['elements'][0]['distance']['text']
    adj_final_api.loc[idx, 'duration'] = res_json['rows'][0]['elements'][0]['duration']['text']

Drop columns we no longer need and save the raw data file.

In [ ]:
adj_final_api = adj_final_api.drop(['origin','dest'], axis = 1)
adj_final_api.head()

In [ ]:
adj_final_api.to_csv('adj_dist_all_raw.csv', index=False)

### 4.2.3 Clean Up API Data

Since the final API file is missing the distance for several pairs of counties, we fill those values in manually and save the file to adj_dist_all_raw_copy.csv. To clean up the data, we start by loading in the raw version (prevents us from running the API code again).

In [35]:
url = "https://raw.githubusercontent.com/AnjaDeric/MDA-TeamCroatia/main/Data/Mid-Points/adj_dist_all_raw_copy.csv"
adj_final_api = pd.read_csv(url)
adj_final_api.head()

,county_name,county_state,county_fips,bcounty_name,bcounty_state,bcounty_fips,county_combined,gc_dist_km,distance,duration
0,Clinton County,PA,42035,Potter County,PA,42105,4203542105,59.5,99.9 km,1 hour 35 mins
1,Duval County,TX,48131,Jim Hogg County,TX,48247,4813148247,73.9,99.9 km,1 hour 22 mins
2,Jefferson Parish,LA,22051,St. Tammany Parish,LA,22103,2205122103,101.1,99.8 km,1 hour 12 mins
3,Cottle County,TX,48101,Dickens County,TX,48125,4810148125,70.9,99.8 km,59 mins
4,Pend Oreille County,WA,53051,Stevens County,WA,53065,5305153065,49.0,99.8 km,1 hour 23 mins


We check to make sure no rows are missing data.

In [36]:
adj_final_api.isna().sum()

county_name        0
county_state       0
county_fips        0
bcounty_name       0
bcounty_state      0
bcounty_fips       0
county_combined    0
gc_dist_km         0
distance           0
duration           0
dtype: int64

Extract number of km from the distance column and convert it into a float format.

In [ ]:
adj_final_api['distance'].replace(regex=True, inplace=True, to_replace=r'[^0-9.\-]', value=r'')
adj_final_api['distance'] = adj_final_api['distance'].astype(float)
adj_final_api.rename(columns={'distance': 'd_dist_km'}, inplace=True)
adj_final_api.head()

,county_name,county_state,county_fips,bcounty_name,bcounty_state,bcounty_fips,county_combined,gc_dist_km,d_dist_km,duration
0,Clinton County,PA,42035,Potter County,PA,42105,4203542105,59.5,99.9,1 hour 35 mins
1,Duval County,TX,48131,Jim Hogg County,TX,48247,4813148247,73.9,99.9,1 hour 22 mins
2,Jefferson Parish,LA,22051,St. Tammany Parish,LA,22103,2205122103,101.1,99.8,1 hour 12 mins
3,Cottle County,TX,48101,Dickens County,TX,48125,4810148125,70.9,99.8,59 mins
4,Pend Oreille County,WA,53051,Stevens County,WA,53065,5305153065,49.0,99.8,1 hour 23 mins


To extract the total time (as a number) from the duration column, we first create a function that takes a string in the format 'X hour Y mins' and converts it into number of minutes. This function was adapted from [this link](https://stackoverflow.com/questions/70736772/how-to-convert-a-hours-and-minutes-and-seconds-string-to-hhmmss-format).

In [ ]:
def duration_to_num(total_time): 
    hh = mm = 0
    for word in total_time.split():
        word = word.lower()
        # save each digit as hh or mm based on the word following the digit
        if word.isdigit():
            save = word
        elif word.startswith('hour'):
            hh = int(save)
        elif word.startswith('mins'):
            mm = int(save)

    # return total number of minutes
    return (hh*60+mm)

We then apply the duration_to_num function to the duration column in all rows of the data frame to get the total number of minutes between each county.

In [ ]:
# using apply function to create a new column
adj_final_api['duration_min'] = adj_final_api.apply(lambda row: duration_to_num(row.duration), axis = 1)
adj_final_api.drop(['duration'], axis=1, inplace=True)
adj_final_api.head()

,county_name,county_state,county_fips,bcounty_name,bcounty_state,bcounty_fips,county_combined,gc_dist_km,d_dist_km,duration_min
0,Clinton County,PA,42035,Potter County,PA,42105,4203542105,59.5,99.9,95
1,Duval County,TX,48131,Jim Hogg County,TX,48247,4813148247,73.9,99.9,82
2,Jefferson Parish,LA,22051,St. Tammany Parish,LA,22103,2205122103,101.1,99.8,72
3,Cottle County,TX,48101,Dickens County,TX,48125,4810148125,70.9,99.8,59
4,Pend Oreille County,WA,53051,Stevens County,WA,53065,5305153065,49.0,99.8,83


In [ ]:
adj_final_api.to_csv('adj_dist_all_final.csv', index=False)